In [1]:
import src.habit as ha
h1 = ha.Habit.get_instance(2)

In [2]:
h1.completed

False

In [3]:
h1.__dict__

{'habit': 'Tea',
 'description': 'drink more Tea',
 'period': 1,
 'duration': 30,
 'created': '2022-11-20 11:11:16.648309',
 'completed': False,
 'closed': 0,
 'last_completion_date': '2022-12-19 19:28:00',
 'is_template': 0,
 'habits_id': 2}

In [4]:
h2 = ha.Habit.get_instance(4)
h2.check_duration()

You have 49 days left to complete the task


41